In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data = pd.read_csv("C:/Users/tupka/Downloads/estimation_with_bmi/generated_data.csv")

In [3]:
X = data[['Age', 'Gender']].values

# Targets: Height, Weight, and BMI
y = data[['Height', 'Weight', 'BMI']].values

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [5]:
class EstimatorModel(nn.Module):
    def __init__(self):
        super(EstimatorModel, self).__init__()
        self.fc1 = nn.Linear(2, 64)  # Reduce the input layer size
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 3)  # Output layer
        self.dropout = nn.Dropout(0.2)  # Dropout layer to prevent overfitting
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        return self.fc4(x)

# Hyperparameters
learning_rate = 0.1
epochs = 1000
# 4. Instantiate the model, loss function, and optimizer
model = EstimatorModel()
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    y_pred = model(X_train_tensor)

    # Compute loss
    loss = criterion(y_pred, y_train_tensor)

    # Backpropagation
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item()}')

Epoch [0/1000], Loss: 11156.615234375
Epoch [100/1000], Loss: 203.02667236328125
Epoch [200/1000], Loss: 153.9930419921875
Epoch [300/1000], Loss: 146.51487731933594
Epoch [400/1000], Loss: 147.16876220703125
Epoch [500/1000], Loss: 146.43418884277344
Epoch [600/1000], Loss: 145.40318298339844
Epoch [700/1000], Loss: 147.14981079101562
Epoch [800/1000], Loss: 145.7273712158203
Epoch [900/1000], Loss: 145.5930633544922


In [7]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    mse = mean_squared_error(y_test_tensor, predictions)
    print(f'Mean Squared Error on Test Set: {mse:.4f}')


Mean Squared Error on Test Set: 141.1610


In [8]:
age = 20
gender = 1  # Female
input_data = torch.tensor([[age, gender]], dtype=torch.float32)

model.eval()
with torch.no_grad():
    predicted = model(input_data)
    print(f"Predicted Height: {predicted[0][0].item():.2f} cm")
    print(f"Predicted Weight: {predicted[0][1].item():.2f} kg")
    print(f"Predicted BMI: {predicted[0][2].item():.2f}")

Predicted Height: 167.83 cm
Predicted Weight: 74.99 kg
Predicted BMI: 26.99


In [ ]:
# Save the model
torch.save(model.state_dict(), 'new_model.pth')
print("Model saved successfully!")
